In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


True

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
811,I've always been enthusiastic about period dra...,negative
713,Brian Keith as Cole Wlikerson and Richard Jaec...,positive
608,If you are in to bad movies for the entertainm...,negative
706,I am from the Dallas/Fort Worth area and lived...,negative
654,I recently saw this movie in my International ...,positive


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
811,always enthusiastic period drama art form bbc ...,negative
713,brian keith cole wlikerson richard jaeckel wad...,positive
608,bad movie entertainment witnessing bad movie b...,negative
706,dallas fort worth area lived arlington year mo...,negative
654,recently saw movie international business clas...,positive


In [9]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
811,always enthusiastic period drama art form bbc ...,0
713,brian keith cole wlikerson richard jaeckel wad...,1
608,bad movie entertainment witnessing bad movie b...,0
706,dallas fort worth area lived arlington year mo...,0
654,recently saw movie international business clas...,1


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/fardinemonbd/MLOps-capstone-project.mlflow')
dagshub.init(repo_owner='fardinemonbd', repo_name='MLOps-capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\User\anaconda3\envs\atlas\lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6097f1b1-7fe0-4689-bc5b-9ad598ef67e1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f98cd00d96a32e9fe91d9299bf94c4c2ead54d02ced91068c59dd8783cd13ec0




Accessing as fardinemonbd

Initialized MLflow to track repo "fardinemonbd/MLOps-capstone-project"

Repository fardinemonbd/MLOps-capstone-project initialized!

2025/06/20 15:37:47 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/18476548f90f44d898783eab7cd7da78', creation_time=1750412266673, experiment_id='0', last_update_time=1750412266673, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-20 16:02:18,076 - INFO - Starting MLflow run...
2025-06-20 16:02:19,790 - INFO - Logging preprocessing parameters...
2025-06-20 16:02:23,796 - INFO - Initializing Logistic Regression model...
2025-06-20 16:02:23,803 - INFO - Fitting the model...
2025-06-20 16:02:23,840 - INFO - Model training complete.
2025-06-20 16:02:23,844 - INFO - Logging model parameters...
2025-06-20 16:02:24,204 - INFO - Making predictions...
2025-06-20 16:02:24,207 - INFO - Calculating evaluation metrics...
2025-06-20 16:02:24,219 - INFO - Logging evaluation metrics...
2025-06-20 16:02:26,756 - INFO - Saving and logging the model...
2025/06/20 16:02:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-06-20 16:02:39,403 - INFO - Model training and logging completed in 19.61 seconds.
2025-06-20 16:02:39,405 - INFO - Accuracy: 0.584
2025-06-20 16:02:39,407 - INFO - Precision: 0.5294117647058824
2025-06